In [1]:
import pandas as pd
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import remove_stopwords
import string
import traceback


In [2]:
import nltk

# Uncomment the line below and run it once to download the necessary data
# nltk.download('punkt')

In [3]:

# Load CSV file into a DataFrame
df = pd.read_csv("bigBasketProducts.csv")
df

,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description
0,1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.00,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
1,2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.00,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
2,3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.00,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."
3,4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.00,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...
4,5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.00,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...
...,...,...,...,...,...,...,...,...,...,...
27550,27551,"Wottagirl! Perfume Spray - Heaven, Classic",Beauty & Hygiene,Fragrances & Deos,Layerr,199.20,249.0,Perfume,3.9,Layerr brings you Wottagirl Classic fragrant b...
27551,27552,Rosemary,Gourmet & World Food,Cooking & Baking Needs,Puramate,67.50,75.0,"Herbs, Seasonings & Rubs",4.0,Puramate rosemary is enough to transform a dis...
27552,27553,Peri-Peri Sweet Potato Chips,Gourmet & World Food,"Snacks, Dry Fruits, Nuts",FabBox,200.00,200.0,Nachos & Chips,3.8,We have taken the richness of Sweet Potatoes (...
27553,27554,Green Tea - Pure Original,Beverages,Tea,Tetley,396.00,495.0,Tea Bags,4.2,"Tetley Green Tea with its refreshing pure, ori..."


In [4]:
# Assuming 'description' is the column containing product descriptions
# Fill NaN values with an empty string
df['description'] = df['description'].fillna("")
df

,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description
0,1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.00,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
1,2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.00,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
2,3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.00,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."
3,4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.00,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...
4,5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.00,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...
...,...,...,...,...,...,...,...,...,...,...
27550,27551,"Wottagirl! Perfume Spray - Heaven, Classic",Beauty & Hygiene,Fragrances & Deos,Layerr,199.20,249.0,Perfume,3.9,Layerr brings you Wottagirl Classic fragrant b...
27551,27552,Rosemary,Gourmet & World Food,Cooking & Baking Needs,Puramate,67.50,75.0,"Herbs, Seasonings & Rubs",4.0,Puramate rosemary is enough to transform a dis...
27552,27553,Peri-Peri Sweet Potato Chips,Gourmet & World Food,"Snacks, Dry Fruits, Nuts",FabBox,200.00,200.0,Nachos & Chips,3.8,We have taken the richness of Sweet Potatoes (...
27553,27554,Green Tea - Pure Original,Beverages,Tea,Tetley,396.00,495.0,Tea Bags,4.2,"Tetley Green Tea with its refreshing pure, ori..."


In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vedan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
# from transformers import GPT2LMHeadModel, GPT2Tokenizer

# # Load pre-trained GPT-2 model and tokenizer
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# model = GPT2LMHeadModel.from_pretrained("gpt2")



In [7]:
import os
PINECONE_API_KEY="ec5d5e0d-f392-4654-ad4c-d030f2f8adc7"
PINECONE_ENV="gcp-starter"

import pinecone
pinecone.init(api_key="your_api_key", environment="your_environment", host="pinecone.io")

# pinecone.init(
#     api_key=os.environ["PINECONE_API_KEY"], environment=os.environ["PINECONE_ENV"]
# )

C:\Users\vedan\AppData\Roaming\Python\Python39\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [8]:
import os
import pinecone

# Set the environment variables
os.environ["PINECONE_API_KEY"] = "ec5d5e0d-f392-4654-ad4c-d030f2f8adc7"
os.environ["PINECONE_ENV"] = "gcp-starter"

# Specify the local Pinecone service URL with the desired port
local_pinecone_url = "http://localhost:5000"  # Replace 12345 with the desired port


pinecone.init(
    api_key=os.environ["PINECONE_API_KEY"], environment=os.environ["PINECONE_ENV"]
)


In [9]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores import Pinecone

# openai_api_key = "ec5d5e0d-f392-4654-ad4c-d030f2f8adc7"  # Replace with your actual API key
# embeddings = OpenAIEmbeddings(api_key=openai_api_key)
# # create new index
# pinecone.create_index("langchain-self-retriever-demo", dimension=1536)
import os
import pinecone

# Set the environment variables
os.environ["PINECONE_API_KEY"] = "ec5d5e0d-f392-4654-ad4c-d030f2f8adc7"
os.environ["PINECONE_ENV"] = "gcp-starter"

# Specify the local Pinecone service URL with the desired port
local_pinecone_url = "https://langchain-self-retriever-demo-2wvunwi.svc.gcp-starter.pinecone.io"  # Replace 12345 with the desired port

# Initialize Pinecone with the local URL
pinecone.init(api_key=os.environ["PINECONE_API_KEY"], environment=os.environ["PINECONE_ENV"], host=local_pinecone_url)

# # Create a new index in Pinecone
# pinecone.create_index("langchain-self-retriever-demo", dimension=1536)


In [10]:
# Assuming you have a DataFrame 'df' with a column 'your_text_column' containing the textual content
docs = [
    Document(
        page_content=row["description"],
        metadata=row.drop("description").to_dict(),
    )
    for _, row in df.iterrows()
]

In [11]:
from langchain.schema import Document
from langchain.vectorstores import Pinecone
from sentence_transformers import SentenceTransformer

# Load a pre-trained sentence transformer model (you can choose other models from sentence-transformers)
model = SentenceTransformer('paraphrase-mpnet-base-v2')

# Define the embedding function using sentence-transformers
def embedding_function(text):
    # Encode the text using the sentence transformer model
    return model.encode(text).tolist()

# Assuming you have a DataFrame 'df' with a column 'description' containing the textual content
docs = [
    Document(
        page_content=row["description"],
        metadata=row.drop("description").to_dict(),
    )
    for _, row in df.iterrows()
]

# Create a Pinecone vector store from the documents
vectorstore = Pinecone.from_documents(
    docs, embedding=embedding_function, index_name="langchain-self-retriever-demo"
)


KeyboardInterrupt: 

In [ ]:
# # Create a Pinecone vector store from the documents
# from langchain.vectorstores import Pinecone
# vectorstore = Pinecone.from_documents(
#     docs, embedding=None, index_name="langchain-self-retriever-demo"
# )

TypeError: 'NoneType' object is not callable